### Get mangroves for all years for AOI and calc extent changes for canopy cover classes

In [1]:
import datacube
import numpy as np
import xarray as xr
import pandas as pd
import os
from datacube import Datacube
from datacube.utils import geometry
import rioxarray
from datacube.utils.cog import write_cog
dc = datacube.Datacube(app="cyclone mangroves")

pd.set_option('display.max_columns', None)


### fill in data (doing one cyclone at a time for the moment)
option to export area of canopy cover classes as csv
option to export geotiff of before and after cyclone mangrove dataset

In [2]:
cyclone_name = 'Yasi'
cyclone_time = 2011

# cyclone_name = 'Larry'
# cyclone_time = 2006

# cyclone_name = 'Ita'
# cyclone_time = 2014

# cyclone_name = 'Marcia'
# cyclone_time = 2015

# cyclone_name = 'Nathan_Landfall1'
# cyclone_time = 2015

export_as_csv = True

export_before_after_cyclone_mangrove_dataset = True

### load in geotiff

In [3]:
geotiff_path = './FINALlocalWind/'+cyclone_name+'_30m_EPSG3577.tif'
# Open into an xarray.DataArray
geotiff_da = rioxarray.open_rasterio(geotiff_path)
# change -9999 to 0 #### not sure this needs to be done, -9999 should be considered 'not cyclone'?
geotiff_da = xr.where(geotiff_da == -9999, 0, geotiff_da.values)
# Covert our xarray.DataArray into a xarray.Dataset
geotiff_ds = geotiff_da.to_dataset('band')
# Rename the variable to a more useful name
dataset = geotiff_ds.rename({1: 'wind_speed'})# get extent of cyclone dataset
geobox = dataset.extent

### load mangrove data on extent of geotiff

In [4]:
# get extent of cyclone dataset
geobox = dataset.extent

In [5]:
# 1987 - 2021 datasets
mangrove_datasets = dc.load(product = 'ga_ls_mangrove_cover_cyear_3', 
                            geopolygon=dataset.extent, 
                            output_crs='EPSG:3577', 
#                             time = ('2010-01-01', '2011-12-31'),
                            resolution=(-30, 30))

### create dataframe and append areas (km2 - multiple each pixel by 0.0009) for each annual mangrove dataset

##### 0 = not observed/missing data
##### 1 = 20-50% canopy cover
##### 2 = 50-80% canopy cover
##### 3 = 80-100% canopy cover
##### 255 =  no mangroves


In [6]:
# set up df with rows for each canopy_cover_class
df = pd.DataFrame([0, 1, 2, 3, 255])

# for each time
for t in mangrove_datasets.canopy_cover_class:
    # get year as string
    time = str(t.time.values)[:4]
    # get canopy_cover_class and pixel counts
    unique, counts = np.unique(t, return_counts=True)
    item = np.asarray((unique, counts)).T
    # check values looks correct
#     print(item)
    # append to df (divide counts by 0.0009 to give area in km2)
    df[time] = pd.Series(counts*0.0009)

In [7]:
df

,0,1987,1988,1989,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,0,0.4680,0.1017,0.6210,0.1125,0.0306,0.0378,0.2016,0.0792,0.0648,0.0855,0.2124,0.2295,0.0783,0.1800,0.0387,0.0558,0.0162,0.0684,0.1476,0.1647,0.6003,0.3411,0.1485,0.1188,0.0198,0.1467,0.1494,0.1953,0.4239,0.2637,0.2079,0.2061,0.2250,0.1980,0.2268
1,1,68.8761,23.3235,26.1306,25.3053,30.5568,55.6551,25.7013,39.5064,32.9670,38.1339,23.0868,33.3981,16.6419,10.8135,17.2359,26.3970,25.6599,28.3599,26.2746,20.6973,24.7032,23.8077,26.2242,14.7150,73.4751,31.7871,22.1760,21.1851,20.0205,20.4228,13.3470,14.5809,20.8638,12.8142,8.9847
2,2,166.3803,186.4152,169.7436,167.4576,172.3644,201.5775,196.4565,225.0378,195.1578,194.4207,161.2683,163.2159,152.3061,109.1025,156.9528,170.4033,162.7983,185.4036,160.3044,160.3071,189.5238,207.6714,190.0719,169.7940,265.6449,196.6482,168.7005,165.4965,154.3392,167.4864,141.6816,149.2425,172.2249,138.3543,111.4569
3,3,193.6782,227.4498,229.7178,244.0026,232.6977,176.0436,212.2659,170.4492,208.8315,205.2963,252.1593,232.7427,271.6209,320.3496,266.1381,244.2690,251.5518,226.2420,252.2790,258.9156,215.0712,203.5611,220.4028,256.4658,100.0917,209.3472,246.2634,248.9580,260.5374,248.9877,284.1615,272.9961,242.3439,286.8975,317.0232
4,255,60144.5142,60136.6266,60147.7038,60137.0388,60138.2673,60140.6028,60139.2915,60138.8442,60136.8957,60135.9804,60137.1900,60144.3306,60133.2696,60133.4712,60133.5513,60132.7917,60133.8906,60133.8429,60134.9112,60133.8321,60144.0183,60138.5355,60137.0694,60132.8232,60134.6853,60135.9876,60136.6275,60138.0819,60138.5958,60136.7562,60134.5188,60136.8912,60138.2592,60135.6528,60136.2252


#### export

In [8]:
# export as csv?
if export_as_csv == True:
    df.to_csv('./'+cyclone_name+'_area_km2.csv')
else:
    pass

In [9]:
# export before and after cyclone datasets
if export_before_after_cyclone_mangrove_dataset == True:
    # get 1 year before and year of cyclone data
    time_of_cyclone = (str(cyclone_time - 1) + '-01-01', str(cyclone_time) + '-12-31')

    # before and after cyclone datasets
    mangrove_datasets = dc.load(product = 'ga_ls_mangrove_cover_cyear_3', 
                                geopolygon=dataset.extent, 
                                output_crs='EPSG:3577', 
                                time = time_of_cyclone,
                                resolution=(-30, 30))
    # for time in canopy_cover_class
    for t in mangrove_datasets.canopy_cover_class:
        # get time as string for filesave
        time = str(t.time.values)[:4]
        print(time)
        write_cog(geo_im=t,
                  fname='./'+cyclone_name+'_mangrove_canopy_cover_'+time+'.tif',
                  overwrite=True)
else:
    pass

2010
2011
